In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
seed = 42

Uso los dataframe de train y test que salen de data prepocesing.

In [5]:
train = pd.read_csv("../datasets/xgb-train.csv")
test = pd.read_csv("../datasets/xgb-test.csv")

In [24]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [29]:
target = "target"
features = list(train.columns)
features.remove(target)
features.remove("Opportunity_ID")

#VANILLA NO
# features.remove("delivery_delay")
# features.remove("opportunity_lifetime")
# features.remove("converted_taxable_amount")
# features.remove("last_modified_to_delivery")
# features.remove("currency_conversion_rate")
# features.remove("Occur")
# features.remove("delivery_window")
# features.remove("account_creation_to_created_opp")

#VANILLA SI
features.remove('Total_Taxable_Amount')
#features.remove('ASP_(converted)')
features.remove("ASP")
features.remove("Total_Amount")
features.remove("Delivery_Year")
# features.remove("Week_Day")

#CONSIDERO REMOVIBLES
# features.remove("created_blocknum")   #FECHA!!
features.remove("late_delivery_blocknum")
features.remove("early_delivery_blocknum")
features.remove("last_modified_blocknum")
features.remove("account_creation_blocknum") #OJO CCON HIPOTESIS DEL CLIENTE VIEEJO
print("Features totales: {}".format(len(features)))


Features totales: 116


In [30]:
X_train, y_train = train.loc[:,features], train.loc[:,target]
X_test_Opp = test.loc[:,"Opportunity_ID"]
X_test = test.loc[:,features]

print("Set de entrenamiento (size) {}".format(X_train.shape))
print("Set de testing (size) {}".format(X_test.shape))

Set de entrenamiento (size) (16947, 116)
Set de testing (size) (2551, 116)


In [31]:
percent_80 = train["created_blocknum"].describe()['75%']/0.945
beta_test = train[train["created_blocknum"]>percent_80]
beta_train = train[train["created_blocknum"]<percent_80]

In [40]:
A_train, b_train = beta_train.loc[:,features], beta_train.loc[:,target]
# A_test_Opp = beta_test.loc[:,"Opportunity_ID"]
A_test, b_test = beta_test.loc[:,features], beta_test.loc[:,target]

print("Set de entrenamiento secundario (size) {}".format(A_train.shape))
print("Set de testing secundario (size) {}".format(A_test.shape))

Set de entrenamiento secundario (size) (14078, 116)
Set de testing secundario (size) (2869, 116)


In [41]:
dtrain = lgb.Dataset(X_train, label=y_train)

In [45]:

nfolds = 7
rounds = 160
early_stopping_rounds = 10

params = {"objective": "binary",
          'colsample_bytree': 0.3,
          'subsample': 1.0,
          'learning_rate': 0.1,
          'max_depth': 6,
          'min_child_weight': 1,
          }
beta_params = params.copy()
beta_params['n_estimators'] = 100


enaable_gridsearch_for_tree = True
enaable_gridsearch_for_sampling = True
enaable_gridsearch_for_learning = True
final_cv = True
enable_parcial_training = True

print("Usando nfolds={},  num_boost_round={}"
      "  y  early_stopping_rounds={}".format(nfolds, rounds, early_stopping_rounds))

Usando nfolds=7,  num_boost_round=160  y  early_stopping_rounds=10


In [46]:

def find_best_param_tuple(dtrain, params, param_tuple):
    # Define initial best params and LogLoss
    min_logloss = float("Inf")
    best_params = None
    for param0, param1 in param_tuple['grid']:
        print("CV with {}={}, {}={}".format(param_tuple['names'][0],
                                 param0,
                                 param_tuple['names'][1],
                                 param1))
        # Update our parameters
        params[param_tuple['names'][0]] = param0
        if param_tuple['names'][1]: params[param_tuple['names'][1]] = param1
        # Run CV
        cv_results = lgb.cv(
            params,
            dtrain,
            num_boost_round=rounds,
            seed=1234,
            nfold=nfolds,
            metrics='logloss',
            early_stopping_rounds=early_stopping_rounds
        )
        # Update best LogLoss
        mean_logloss = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLL {} for {} rounds".format(mean_logloss, boost_rounds+1))
        if mean_logloss < min_logloss:
            min_logloss = mean_logloss
            best_params = (param0, param1)
    print("Best {},{}: {}, {}, LogLoss: {}".format(param_tuple['names'][0],param_tuple['names'][1],best_params[0], best_params[1], min_logloss))
    return best_params

In [47]:
if enaable_gridsearch_for_tree:

    tree_params_tuple = {
        'names': ('max_depth', 'min_child_weight'),
        'grid': [
            (max_depth, min_child_weight)
            for max_depth in range(20,26,1)
            for min_child_weight in range(1,2)
        ]
    }

    best_tree_params = find_best_param_tuple(dtrain, params, tree_params_tuple)

    params['max_depth'] = best_tree_params[0]
    params['min_child_weight'] = best_tree_params[1]

CV with max_depth=20, min_child_weight=1
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 8172, number of negative: 6354
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2383
[LightGBM] [Info] Number of data points in the train set: 14526, number of used features: 112
[LightGBM] [W

ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [50]:
gridParams = { 
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [16,32, 64], 
    'random_state' : [501],
    'num_boost_round' : [3000],
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4], 
    }

lgb_estimator = lgb.LGBMClassifier(boosting_type = 'gbdt', 
                                   n_estimators=500, 
                                   learning_rate =  0.05, num_leaves =  64,  
                                   eval_metric  = 'logloss',
                                   verbose_eval=20, 
                                   early_stopping_rounds=10)

g_lgbm = GridSearchCV(estimator=lgb_estimator, param_grid=gridParams, n_jobs = 3, cv= 3)

lgb_model = g_lgbm.fit(X=X_train, y=y_train)

/Users/UltraTuron/Desktop/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/UltraTuron/Desktop/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [66]:
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=seed)
lgb_train = lgb.Dataset(X_train, y_train,
                        free_raw_data=False)
#lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,
 #                   free_raw_data=False)
num_train, num_feature = X_train.shape
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
feature_name = ['feature_' + str(col) for col in range(num_feature)]

print('Starting training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=lgb_train,  # eval training data
                feature_name=feature_name,
                categorical_feature=[21])

print('Finished first 10 rounds...')
# check feature name
print('7th feature name is:', lgb_train.feature_name[6])
print('Saving model...')
# save model to file
gbm.save_model('model.txt')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model='model.txt'
               )
#valid_sets=lgb_eva
print('Finished 10 - 20 rounds with model file...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                learning_rates=lambda iter: 0.05 * (0.99 ** iter)
               )

print('Finished 20 - 30 rounds with decay learning rates...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                callbacks=[lgb.reset_parameter(bagging_fraction=[0.7] * 5 + [0.6] * 5)])

print('Finished 30 - 40 rounds with changing bagging_fraction...')
def loglikelihood(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess
def binary_error(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    return 'error', np.mean(labels != (preds > 0.5)), False

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                fobj=loglikelihood,
                feval=binary_error
               )

print('Finished 40 - 50 rounds with self-defined objective function and eval metric...')
def accuracy(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    return 'accuracy', np.mean(labels == (preds > 0.5)), True


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                fobj=loglikelihood,
                feval=[binary_error, accuracy]
               )

print('Finished 50 - 60 rounds with self-defined objective function '
      'and multiple self-defined eval metrics...')

Starting training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	training's binary_logloss: 0.659351
[2]	training's binary_logloss: 0.635554
[3]	training's binary_logloss: 0.613965
[4]	training's binary_logloss: 0.5938
[5]	training's binary_logloss: 0.576552
[6]	training's binary_logloss: 0.559164
[7]	training's binary_logloss: 0.543011
[8]	training's binary_logloss: 0.527743
[9]	training's binary_logloss: 0.513743
[10]	training's binary_logloss: 0.500676
Finished first 10 rounds...
7th feature name is: feature_6
Saving model...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016759 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Finished 10 - 20 rounds with model file...
[Lig